In [1]:
import math
import numpy as np

In [2]:
# Parâmetros do robô
diametro_rodas = 5  # cm
comprimento_eixo = 20  # cm
raio_rodas = diametro_rodas / 2
resolucao_grid = 10  # cm

In [3]:
# Velocidade angular máxima das rodas
velocidade_angular_maxima = 10  # rpm
velocidade_angular_maxima_rad_s = 2 * math.pi * velocidade_angular_maxima / 60

In [4]:
# Definindo as posições iniciais e finais
pose_inicial = (0, 0, 0)  # (X, Y, Theta) em cm e graus
pose_intermediaria = (4, 6, 45)  # (X, Y, Theta) em cm e graus
pose_final = (6, 8, 90)  # (X, Y, Theta) em cm e graus

In [5]:
# Função para calcular a matriz cinemática
def calcular_matriz_cinemática(velocidade_linear, velocidade_angular, comprimento_eixo):
    matriz_cinemática = np.array([
        [raio_rodas / 2, raio_rodas / 2],
        [raio_rodas / comprimento_eixo, -raio_rodas / comprimento_eixo]
    ])
    velocidades_roda = np.array([velocidade_linear, velocidade_angular])
    velocidades = np.dot(matriz_cinemática, velocidades_roda)
    return velocidades

In [7]:
# Função para planejar o movimento do robô
def planejar_movimento(pose_inicial, pose_final, resolucao_grid, velocidade_angular_maxima_rad_s):
    x_inicial, y_inicial, theta_inicial = pose_inicial
    x_final, y_final, theta_final = pose_final

    tempo_total = 0
    movimentos = []

    while True:
        # Calcular a diferença entre as posições inicial e final
        dx = x_final - x_inicial
        dy = y_final - y_inicial
        dtheta = math.radians(theta_final - theta_inicial)

        # Calcular a velocidade linear e angular necessárias
        distancia = math.sqrt(dx**2 + dy**2)
        tempo = distancia / (resolucao_grid / 100)  # Converter resolução para metros
        velocidade_linear = distancia / tempo
        velocidade_angular = dtheta / tempo

        # Verificar se a velocidade angular não excede a máxima
        if abs(velocidade_angular) > velocidade_angular_maxima_rad_s:
            velocidade_angular = math.copysign(velocidade_angular_maxima_rad_s, velocidade_angular)

        # Adicionar o movimento à lista
        movimentos.append((velocidade_linear, math.degrees(velocidade_angular)))

        # Atualizar a posições inicial
        x_inicial = x_final
        y_inicial = y_final
        theta_inicial = theta_final

        tempo_total += tempo

        # Verificar se alcançamos a posições final
        if x_inicial == x_final and y_inicial == y_final and theta_inicial == theta_final:
            break

    return movimentos, tempo_total

In [8]:
# Planejar o movimento do robô
movimentos, tempo_total = planejar_movimento(pose_inicial, pose_intermediaria, resolucao_grid, velocidade_angular_maxima_rad_s)
movimentos2, tempo_total2 = planejar_movimento(pose_intermediaria, pose_final, resolucao_grid, velocidade_angular_maxima_rad_s)

# Imprimir os movimentos planejados
print("Movimentos para a pose intermediária:")
for movimento in movimentos:
    print(f"Velocidade Linear: {movimento[0]:.2f} m/s, Velocidade Angular: {movimento[1]:.2f} rad/s")

print("\nMovimentos para a pose final:")
for movimento in movimentos2:
    print(f"Velocidade Linear: {movimento[0]:.2f} m/s, Velocidade Angular: {movimento[1]:.2f} rad/s")

print(f"\nTempo total para alcançar a pose intermediária: {tempo_total:.2f} segundos")
print(f"Tempo total para alcançar a pose final: {tempo_total2:.2f} segundos")

Movimentos para a pose intermediária:
Velocidade Linear: 0.10 m/s, Velocidade Angular: 0.62 rad/s

Movimentos para a pose final:
Velocidade Linear: 0.10 m/s, Velocidade Angular: 1.59 rad/s

Tempo total para alcançar a pose intermediária: 72.11 segundos
Tempo total para alcançar a pose final: 28.28 segundos
